In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/29 16:42:56 WARN Utils: Your hostname, Silvialaptop resolves to a loopback address: 127.0.1.1; using 172.31.73.54 instead (on interface eth0)
24/03/29 16:42:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/29 16:42:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

</br>
SELECT </br>
    date_trunc('hour', lpep_pickup_datetime) AS hour, </br>
    PULocationID AS zone,</br>
</br>
    SUM(total_amount) AS amount,</br>
    COUNT(1) AS number_records</br>
FROM</br>
    green</br>
WHERE</br>
    lpep_pickup_datetime >= '2020-01-01 00:00:00'</br>
GROUP BY</br>
    1, 2</br>
</br>

In [3]:
df_green

DataFrame[VendorID: int, lpep_pickup_datetime: timestamp, lpep_dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: int, PULocationID: int, DOLocationID: int, passenger_count: int, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, ehail_fee: double, improvement_surcharge: double, total_amount: double, payment_type: int, trip_type: int, congestion_surcharge: double]

In [11]:
df_green.rdd

MapPartitionsRDD[17] at javaToPython at NativeMethodAccessorImpl.java:0

In [9]:
df_green.rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 6, 19, 46, 3), lpep_dropoff_datetime=datetime.datetime(2020, 1, 6, 19, 51, 46), store_and_fwd_flag='N', RatecodeID=1, PULocationID=146, DOLocationID=7, passenger_count=1, trip_distance=1.07, fare_amount=6.0, extra=1.0, mta_tax=0.5, tip_amount=1.56, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=9.36, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 14, 39, 3), lpep_dropoff_datetime=datetime.datetime(2020, 1, 5, 14, 47, 52), store_and_fwd_flag='N', RatecodeID=1, PULocationID=97, DOLocationID=49, passenger_count=1, trip_distance=1.27, fare_amount=7.5, extra=0.0, mta_tax=0.5, tip_amount=1.66, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=9.96, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 10, 36, 1), lpep_dropoff_datetim

In [12]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [13]:
rdd.filter(lambda row: True).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 6, 19, 46, 3), PULocationID=146, total_amount=9.36)]

In [14]:
from datetime import datetime

In [15]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [16]:
rows = rdd.take(10)
row = rows[0]

In [17]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 6, 19, 46, 3), PULocationID=146, total_amount=9.36)

In [20]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [21]:
rdd\
    .filter(filter_outliers)\
    .map(prepare_for_grouping)\
    .take(10)

[((datetime.datetime(2020, 1, 6, 19, 0), 146), (9.36, 1)),
 ((datetime.datetime(2020, 1, 5, 14, 0), 97), (9.96, 1)),
 ((datetime.datetime(2020, 1, 5, 10, 0), 7), (9.8, 1)),
 ((datetime.datetime(2020, 1, 6, 10, 0), 42), (6.3, 1)),
 ((datetime.datetime(2020, 1, 6, 13, 0), 75), (17.3, 1)),
 ((datetime.datetime(2020, 1, 1, 14, 0), 41), (5.8, 1)),
 ((datetime.datetime(2020, 1, 4, 11, 0), 74), (13.25, 1)),
 ((datetime.datetime(2020, 1, 3, 16, 0), 130), (12.36, 1)),
 ((datetime.datetime(2020, 1, 4, 14, 0), 66), (4.37, 1)),
 ((datetime.datetime(2020, 1, 5, 14, 0), 159), (8.3, 1))]

In [22]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [23]:
rdd\
    .filter(filter_outliers)\
    .map(prepare_for_grouping)\
    .reduceByKey(calculate_revenue)\
    .take(10)

[((datetime.datetime(2020, 1, 4, 14, 0), 66), (592.51, 23)),
 ((datetime.datetime(2020, 1, 2, 21, 0), 179), (112.00999999999999, 5)),
 ((datetime.datetime(2020, 1, 2, 14, 0), 31), (23.16, 1)),
 ((datetime.datetime(2020, 1, 5, 20, 0), 74), (1089.2099999999994, 65)),
 ((datetime.datetime(2020, 1, 3, 23, 0), 146), (19.1, 2)),
 ((datetime.datetime(2020, 1, 6, 18, 0), 75), (1182.4699999999987, 88)),
 ((datetime.datetime(2020, 1, 1, 1, 0), 210), (56.92999999999999, 3)),
 ((datetime.datetime(2020, 1, 2, 17, 0), 82), (810.2699999999998, 51)),
 ((datetime.datetime(2020, 1, 4, 9, 0), 74), (505.66000000000037, 41)),
 ((datetime.datetime(2020, 1, 4, 18, 0), 28), (48.36, 1))]

In [24]:
from collections import namedtuple

In [25]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [26]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [27]:
from pyspark.sql import types

In [28]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [29]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [30]:
df_result.write.parquet('tmp/green-revenue')

24/03/29 16:01:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [6]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [38]:
df_green \
    .select(columns) \
    .show()

+--------+--------------------+------------+------------+-------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|
+--------+--------------------+------------+------------+-------------+
|       2| 2020-01-06 19:46:03|         146|           7|         1.07|
|       2| 2020-01-05 14:39:03|          97|          49|         1.27|
|       2| 2020-01-05 10:36:01|           7|         223|         2.31|
|       2| 2020-01-06 10:31:58|          42|          41|         0.89|
|       2| 2020-01-06 13:11:19|          75|          42|          2.5|
|       2| 2020-01-01 14:49:16|          41|          42|         0.83|
|       1| 2020-01-04 11:03:20|          74|         236|          1.7|
|       2| 2020-01-03 16:42:38|         130|         215|         1.63|
|       1| 2020-01-04 14:37:06|          66|          66|          0.3|
|       2| 2020-01-05 14:11:12|         159|         147|         1.39|
|       2| 2020-01-04 15:13:54|         166|         151|       

In [47]:
def in_batch(partition):
    return [1]

In [51]:
duration_rdd \
    .mapPartitions(in_batch)\
    .collect()

[1, 1, 1, 1, 1, 1, 1, 1]

In [52]:
def in_batch2(partition):
    cnt = 0
    for row in partition:
        cnt += 1
    return [cnt]

In [53]:
duration_rdd \
    .mapPartitions(in_batch2)\
    .collect()

[1468442, 608803, 480967, 474617, 469612, 443925, 381811, 250517]

In [7]:
import pandas as pd

In [8]:
rows = duration_rdd.take(10)

In [9]:
df = pd.DataFrame(rows, columns=columns)

In [10]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [54]:
df

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2,2020-01-06 19:46:03,146,7,1.07
1,2,2020-01-05 14:39:03,97,49,1.27
2,2,2020-01-05 10:36:01,7,223,2.31
3,2,2020-01-06 10:31:58,42,41,0.89
4,2,2020-01-06 13:11:19,75,42,2.50
5,2,2020-01-01 14:49:16,41,42,0.83
6,1,2020-01-04 11:03:20,74,236,1.70
7,2,2020-01-03 16:42:38,130,215,1.63
8,1,2020-01-04 14:37:06,66,66,0.30
9,2,2020-01-05 14:11:12,159,147,1.39


In [11]:
def in_batch3(partition):
    df = pd.DataFrame(rows, columns=columns)
    cnt = len(df)
    return [cnt]

In [12]:
duration_rdd \
    .mapPartitions(in_batch3)\
    .collect()

/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import

[10, 10, 10, 10, 10, 10, 10, 10]

In [13]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [14]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [15]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/cyxsilvia/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [16]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|5.3500000000000005|
|              6.35|
|             11.55|
|              4.45|
|              12.5|
|4.1499999999999995|
|               8.5|
| 8.149999999999999|
|               1.5|
| 6.949999999999999|
|               2.7|
|              7.25|
|5.6000000000000005|
|               5.5|
|             16.15|
|              12.1|
|               9.9|
|             35.45|
| 4.699999999999999|
|             16.15|
+------------------+
only showing top 20 rows

